Make an MT5 indicator that highlights the corresponding candle red if bearish and blue if bullish on the very first candlestick formed at 0:00. Use MQL5, do not ever mix MT4-style functions in MT5, do not ever use the MT4-style helper. 

#property indicator_chart_window
#property indicator_buffers 5
#property indicator_plots   1

#property indicator_label1  "Midnight Candle"
#property indicator_type1   DRAW_COLOR_CANDLES
#property indicator_color1  clrBlue, clrRed
#property indicator_width1  2

double OpenBuf[];
double HighBuf[];
double LowBuf[];
double CloseBuf[];
double ColorBuf[];

int OnInit()
{
   // Buffers
   SetIndexBuffer(0, OpenBuf,  INDICATOR_DATA);
   SetIndexBuffer(1, HighBuf,  INDICATOR_DATA);
   SetIndexBuffer(2, LowBuf,   INDICATOR_DATA);
   SetIndexBuffer(3, CloseBuf, INDICATOR_DATA);
   SetIndexBuffer(4, ColorBuf, INDICATOR_COLOR_INDEX);

   PlotIndexSetInteger(0, PLOT_COLOR_INDEXES, 2);

   // === Chart Styling ===
   ChartSetInteger(0, CHART_COLOR_BACKGROUND, clrBlack);
   ChartSetInteger(0, CHART_COLOR_FOREGROUND, clrYellow);
   ChartSetInteger(0, CHART_COLOR_GRID, clrBlack);
   ChartSetInteger(0, CHART_SHOW_GRID, false);

   ChartSetInteger(0, CHART_SHOW_BID_LINE, true);
   ChartSetInteger(0, CHART_SHOW_ASK_LINE, true);

   ChartRedraw();

   return(INIT_SUCCEEDED);
}

int OnCalculate(
   const int rates_total,
   const int prev_calculated,
   const datetime &time[],
   const double &open[],
   const double &high[],
   const double &low[],
   const double &close[],
   const long &tick_volume[],
   const long &volume[],
   const int &spread[]
)
{
   if(rates_total < 2)
      return 0;

   int start = prev_calculated > 0 ? prev_calculated - 1 : 0;

   for(int i = start; i < rates_total; i++)
   {
      OpenBuf[i]  = open[i];
      HighBuf[i]  = high[i];
      LowBuf[i]   = low[i];
      CloseBuf[i] = close[i];
      ColorBuf[i] = EMPTY_VALUE;

      MqlDateTime t;
      TimeToStruct(time[i], t);

      // Identify first candle of the day (00:00 open)
      if(t.hour == 0 && t.min == 0)
      {
         if(close[i] > open[i])
            ColorBuf[i] = 0; // Bullish -> Blue
         else if(close[i] < open[i])
            ColorBuf[i] = 1; // Bearish -> Red
      }
   }

   return rates_total;
}
